In [ ]:
import pandas as pd

In [5]:
import praw

reddit = praw.Reddit(client_id='DiypC0N9YzVzaw',
                     client_secret='aQ__rgRY_5H2PdeEGvgIBS9tSUo',
                     password='SlimeySocks925',
                     user_agent='hhh_scraper by u/mr_stephen',
                     username='mr_stephen')

print(reddit.user.me())

mr_stephen


In [17]:
counter = 0
for post in reddit.subreddit('hiphopheads').top("week"):
    counter += 1
    print(post.title)
    if counter == 25:
        break

The Immortal Legacy of Seba Jun, known to many as Nujabes, a legendary Japanese hip-hop producer who kickstarted what could be considered the modern lo-fi movement, alongside Dilla, Madlib, and others. I've spent the last 9 months interviewing those close to him, for further insight.
Pusha-T - The Story of Adidon
[FRESH ALBUM] Denzel Curry - ZUU
[LEAK] KANYE WEST - LIVING SO ITALIAN
Freddie Gibbs & Madlib - Bandana will be released June 28
Tyler, The Creator's "IGOR" sells 172K First Week (78K Pure). DJ Khaled's "Father Of Asahd" sells 132K First Week (32K Pure). Megan Thee Stallion's "Fever" sells 27K First Week (4K Pure).
Kanye reveals his admiration for Andy Kaufman, explains why he doesn't take medicine for his bi-polar, and speaks out again liberal bullies in 60-minute interview with Letterman
Lil Nas X preforms Old Town Road for some super hyped Elementary Schoolers
WWE's John Cena Finally Admits To Stealing "U Can't See Me" & Thuganomics Gimmick From Tony Yayo, 50 Cent & G-Unit


## Collect posts using pushshift.io

Using the `requests` package and [pushshift.io](pushshift.io), which archives reddit posts and comments, I can pull posts from any time period and now worry about the API limits of `praw`.

In [68]:
base_url = "https://api.pushshift.io/reddit/submission/search/"

params = dict()
params['after'] = 1514764801
params['subreddit'] = 'hiphopheads'
params['size'] = 10
r = requests.get(base_url, params = params)
print(r.url)

https://api.pushshift.io/reddit/submission/search/?after=1514764801&subreddit=hiphopheads&size=10


There is a limit of 1000 posts per request, so in order to get all the posts, I'll need to use a loop.

In the loop, I'm going to query the data I want from the API, append it to a list, and then update the `after` paramet to the UTC time of the most recent post in that query. This will effectively let the enw query pick up where the old one left off.

A few things that could be done better here:
* This could be done in parallel. There's a lot of I/O computation waiting for the data to be returned after the request is made. We could pull this with multiple workers. Keep in mind that there's a rate limit of 200 requests per minute using pushshift.io.

* 

In [98]:
posts = []
base_url = "https://api.pushshift.io/reddit/submission/search/"

params = dict()
params['after'] = 1262304001
params['subreddit'] = 'hiphopheads'
params['size'] = 1000

start_time_epoch = time.time()

counter = 0
while params['after'] < start_time_epoch:
    


    r = requests.get(base_url, params = params)
    posts = posts + r.json()['data']
    params['after'] = r.json()['data'][-1]['created_utc']
    
    if counter % 25 == 0:
        print("Making request number", str(counter))
        print("The most recent post was submitted", time.ctime(params['after']))
        print('\n')
        
    counter += 1
    
print("done")

Making request number 0
The most recent post was submitted Wed Jul  7 23:18:27 2010


Making request number 25
The most recent post was submitted Wed Nov 23 18:46:22 2011


Making request number 50
The most recent post was submitted Sat Sep 15 06:21:21 2012


Making request number 75
The most recent post was submitted Thu Feb 28 14:17:52 2013


Making request number 100
The most recent post was submitted Thu Jun  6 16:41:55 2013


Making request number 125
The most recent post was submitted Thu Aug 22 21:04:00 2013


Making request number 150
The most recent post was submitted Thu Nov 14 21:30:03 2013


Making request number 175
The most recent post was submitted Tue Feb 11 00:00:21 2014


Making request number 200
The most recent post was submitted Tue May 13 07:39:16 2014


Making request number 225
The most recent post was submitted Mon Aug 25 10:32:08 2014


Making request number 250
The most recent post was submitted Fri Dec  5 12:53:26 2014


Making request number 275
The most re

In [96]:
pd.DataFrame.from_records(posts).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269420 entries, 0 to 269419
Data columns (total 92 columns):
all_awardings                    15782 non-null object
archived                         19 non-null object
author                           269420 non-null object
author_cakeday                   757 non-null object
author_created_utc               14 non-null float64
author_flair_background_color    55459 non-null object
author_flair_css_class           60659 non-null object
author_flair_richtext            252664 non-null object
author_flair_template_id         21058 non-null object
author_flair_text                1240 non-null object
author_flair_text_color          55459 non-null object
author_flair_type                252664 non-null object
author_fullname                  143187 non-null object
author_id                        5913 non-null object
author_patreon_flair             112857 non-null object
banned_by                        1087 non-null object
brand_safe    